In [4]:
import os
import json
import re
import shutil
from pathlib import Path

def classify_json_version(json_data):
   """JSON 버전 판별"""
   try:
       if "meta_data" in json_data and "annotations" in json_data:
           return "new"
       elif "info" in json_data and "segments" in json_data:
           return "old"
       return "unknown"
   except:
       return "unknown"

def extract_user_num_from_caption(caption):
   """caption에서 user_num 추출"""
   try:
       # 숫자 추출
       numbers = re.findall(r'\d+', caption)
       if numbers:
           valid_numbers = [int(n) for n in numbers if int(n) < 10]
           if valid_numbers:
               return max(valid_numbers)
       
       # 띄어쓰기 제거하고 소문자로 변환하여 검사
       caption_normalized = caption.replace(" ", "").lower()
       
       # 1명으로 판단되는 경우
       single_patterns = ['혼자', '개인', '단독', '일인', '1인']
       if any(pattern in caption_normalized for pattern in single_patterns):
           return 1
           
       # 2명으로 판단되는 경우
       double_patterns = ['둘이', '두명', '두인', '이인', '커플', '부부', '모자', '부자', '남매', '자매', '형제']
       if any(pattern in caption_normalized for pattern in double_patterns):
           return 2
           
       # 3명으로 판단되는 경우
       triple_patterns = ['셋이', '세명', '삼인', '3인']
       if any(pattern in caption_normalized for pattern in triple_patterns):
           return 3
           
       # 4명으로 판단되는 경우
       four_patterns = ['넷이', '네명', '사인', '4인']
       if any(pattern in caption_normalized for pattern in four_patterns):
           return 4
           
       # 다수로 판단되는 경우 (5명으로 설정)
       multi_patterns = ['가족', '단체', '그룹', '다수', '여러명', '여럿이', '많은', '복수']
       if any(pattern in caption_normalized for pattern in multi_patterns):
           return 5
       
       # 기본값
       return 1

   except Exception as e:
       print(f"Error extracting user_num from caption: {str(e)}")
       return 1

def convert_json_to_latest_version(old_json):
   """이전 버전 JSON을 최신 버전으로 변환"""
   try:
       # user_num 결정 (모든 caption 검사)
       max_user_num = 1
       for segment in old_json["segments"]:
           if "caption" in segment:
               user_num = extract_user_num_from_caption(segment["caption"])
               max_user_num = max(max_user_num, user_num)

       new_json = {
           "meta_data": {
               "file_name": old_json["info"]["filename"],
               "format": old_json["info"]["format"],
               "size": old_json["info"]["size"],
               "width_height": old_json["info"]["width_height"],
               "environment": 0,  # 고정값으로 변경
               "frame_rate": 15,  # 고정값
               "total_frames": int(old_json["info"]["playtime"] * 15) if old_json["info"]["playtime"] else 0,
               "camera_height": 170,  # 고정값
               "camera_angle": 15  # 고정값
           },
           "additional_info": old_json["additional_info"],
           "annotations": {
               "space_context": "",
               "user_num": max_user_num,
               "target_objects": [
                   {
                       "object_id": i,
                       "age": 1,
                       "gender": 1,
                       "disability": 2
                   }
                   for i in range(max_user_num)
               ],
               "segmentation": [
                   {
                       "segment_id": seg["segment_id"],
                       "action_type": seg["action"],
                       "start_frame": seg["start_frame"],
                       "end_frame": seg["end_frame"],
                       "duration": seg["end_frame"] - seg["start_frame"],
                       "keyframe": (seg["start_frame"] + seg["end_frame"]) // 2,
                       "keypoints": [
                           {
                               "object_id": i,
                               "keypoints": []
                           }
                           for i in range(max_user_num)
                       ]
                   }
                   for seg in old_json["segments"]
               ]
           }
       }

       return new_json

   except Exception as e:
       print(f"Error converting JSON: {str(e)}")
       raise

def process_folder(input_folder_path):
    """폴더 처리 메인 함수"""
    input_folder = Path(input_folder_path)
    
    # JSON 파일 분류
    old_version_files = []
    new_version_files = []
    
    for json_file in input_folder.glob("*.json"):
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                json_data = json.load(f)
            version = classify_json_version(json_data)
            
            if version == "old":
                old_version_files.append(json_file)
            elif version == "new":
                new_version_files.append(json_file)
        except Exception as e:
            print(f"Error processing {json_file}: {str(e)}")
            continue
    
    # 변환할 파일이 없는 경우 폴더 생성하지 않고 종료
    if not old_version_files:
        print("json이 모두 최신 버전입니다. 변환할 파일이 없습니다")
        return
    
    # 변환할 파일이 있는 경우에만 결과 폴더 생성
    result_folder = input_folder.parent / f"{input_folder.name}_new_ver"
    now_folder = result_folder / "now"
    old_folder = result_folder / "old"
    
    # 이전 결과 폴더가 있다면 삭제
    if result_folder.exists():
        shutil.rmtree(result_folder)
    
    # 폴더 생성
    now_folder.mkdir(parents=True)
    old_folder.mkdir(parents=True)
    
    # 최신 버전 파일 복사
    for file in new_version_files:
        shutil.copy2(file, now_folder / file.name)
    
    # 이전 버전 파일 변환 및 저장
    for file in old_version_files:
        try:
            with open(file, 'r', encoding='utf-8') as f:
                old_data = json.load(f)
            
            new_data = convert_json_to_latest_version(old_data)
            
            with open(old_folder / file.name, 'w', encoding='utf-8') as f:
                json.dump(new_data, f, ensure_ascii=False, indent=2)
                
        except Exception as e:
            print(f"Error converting {file}: {str(e)}")
            continue
    
    print(f"처리 완료:")
    print(f"- 최신 버전 파일 수: {len(new_version_files)}")
    print(f"- 변환된 파일 수: {len(old_version_files)}")
    print(f"결과 저장 위치: {result_folder}")

def get_folder_path():
   while True:
       folder_path = input("처리할 폴더 경로를 입력하세요: ").strip()
       
       # 경로에 따옴표가 있다면 제거
       folder_path = folder_path.strip('"').strip("'")
       
       if not folder_path:
           print("경로를 입력해주세요.")
           continue
           
       path = Path(folder_path)
       if not path.exists():
           print("존재하지 않는 경로입니다.")
           continue
       if not path.is_dir():
           print("폴더가 아닙니다.")
           continue
           
       return str(path)

# 메인 실행 부분
if __name__ == "__main__":
   folder_path = get_folder_path()
   process_folder(folder_path)

처리 완료:
- 최신 버전 파일 수: 0
- 변환된 파일 수: 41
결과 저장 위치: /Users/joyong-eun/Desktop/무제 폴더 2/241105_new_ver
